In [1]:
from Neuroradiomics.registration import *
from Neuroradiomics.skull_stripping import *
from Neuroradiomics.normalization import *
from Neuroradiomics.resampler import *


import itk
import numpy as np
import os
from itkwidgets import view
import sys

import tkinter as tk
from tkinter import filedialog

In [2]:
root = tk.Tk()
root.withdraw()

''

In [19]:
# flair_filename = filedialog.askopenfilename(title = "Select the flair image")

t1_filename = filedialog.askopenfilename(title = "Select the t1 image")

fsl_mask_filename = filedialog.askopenfilename(title = "Select the FSL mask image")

#atlas_filename = filedialog.askopenfilename(title = "Select the atlas image")

#mask_filename = filedialog.askopenfilename(title = "Select the brain mask image")

atlas_filename = '/home/nicolas/Desktop/Neuroradiomics/mni_icbm152_nlin_sym_09a/mni_icbm152_t1_tal_nlin_sym_09a.nii'
mask_filename = '/home/nicolas/Desktop/Neuroradiomics/mni_icbm152_nlin_sym_09a/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii'

t1, fsl_mask = registration_reader(t1_filename, fsl_mask_filename)
atlas, mask = registration_reader(atlas_filename, mask_filename)


In [ ]:
'''

rigid_reg_obj = elastix_rigid_registration(flair, t1)
new_params = Set_sampler_parameters_as_image(rigid_reg_obj.GetTransformParameterObject(), t1)

reg_t1 = itk.transformix_filter(t1, new_params)  

In [20]:
brain_mask = skull_stripping_mask(t1, atlas, mask)
evaluate_stripping(brain_mask, fsl_mask)

The Multimap Registration is done!
Your brain mask is ready!
Dice_Coefficient = 0.9388570393558543
Volume_Similarity = -0.12091376316896432
Hausdorff_Distance = 14.096921369706768
Average_Hausdorff_Distance = 0.13493892805979665


[0.9388570393558543,
 -0.12091376316896432,
 14.096921369706768,
 0.13493892805979665]

In [21]:
dir_path = './56_2016'

if not os.path.exists(dir_path):
        os.mkdir(dir_path)

In [22]:
f = open(dir_path + '/measurements.txt', 'w')
original_stdout = sys.stdout  
sys.stdout = f # Change the standard output to the file we created.
evaluate_stripping(brain_mask, fsl_mask)
sys.stdout = original_stdout # Reset the standard output to its original value
f.close()

In [ ]:
#itk.imwrite(brain_mask, './1_2020/brain_mask.nii')

In [23]:
brain = negative_3d_masking(t1, brain_mask)

In [24]:
matched_fsl = match_physical_spaces(fsl_mask, brain_mask)
matched_fsl.Update()

fsl_brain = negative_3d_masking(t1, matched_fsl.GetOutput() )

In [25]:
itk.imwrite(brain, dir_path + '/brain.nii')

In [26]:
itk.imwrite(fsl_brain, dir_path + '/fsl_brain.nii')